In [92]:
!pip install llama-index

In [93]:
!pip install --upgrade llama-index

In [94]:
!pip install --upgrade llama-index

In [95]:
!pip install langchain

In [96]:
!pip install langchain_community

In [97]:
!pip install --upgrade llama-index

In [101]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

#OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieving documents from the index
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combining the texts of the top 5 results
        return "\n".join([result.text.replace("Patient", "Other Patient") for result in search_results[:5]])

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
    input_variables=["transcript", "knowledge_base"],
    template=(
        "The following is a clinical conversation between a doctor and a patient. "
        "Based on this conversation transcript and using relevant information from the provided knowledgebase, "
        "generate a detailed History of Present Illness (HPI) for the patient. The HPI should include the following sections:\n\n"
        "1. Chief Complaint: Brief description of only the primary symptom.Dont mention the age of the patient.\n"
        "2. Previous Occurrences: Any similar past episodes.\n"
        "3. Family History: Relevant family medical history.\n"
        "4. Symptoms: only current symptoms.\n\n"
        "Conversation Transcript:\n{transcript}\n\n"
        "Relevant Knowledge Base:\n{knowledge_base}\n\n"
        "Please ensure the HPI is comprehensive, clear, and formatted properly."
    )
)




        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)


transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)

<ipython-input-101-7324b6b5b4e4>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)



History of Present Illness (HPI):

Chief Complaint: Stomach pain.

Previous Occurrences:
The patient reports that last night, after a particularly stressful day at work, they ordered Chinese food for dinner. After eating fried rice, fried cranberry, and sweet and sour chicken, the patient started experiencing stomach pain that began in the middle of their abdomen and then moved to the right side. The pain was initially mild and the patient took Tums, thinking it was just indigestion. However, the pain continued to worsen throughout the night and into the next day.

Family History:
The patient's mother has a history of gallbladder attacks and has difficulty eating certain foods. No other relevant family medical history is reported.

Symptoms:
The patient is currently experiencing sharp stomach pain that has worsened since last night. The pain initially started in the middle of their abdomen and then moved to the right side. The patient also reports feeling nauseated and has had some di

In [98]:
#ROS

In [123]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieve documents from the index using the query
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combine the texts of the top results, limiting the total length
        combined_text = ""
        for result in search_results:
            combined_text += result.text.replace("Patient", "Other Patient") + "\n"
            if len(combined_text.split()) > 3000:  # Adjust this limit as needed
                break

        return combined_text

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
            input_variables=["transcript", "knowledge_base"],
            template=(
                "Based on the provided clinical conversation and the following transcript, generate a detailed Review of Systems (RoS) for the patient. "
                "The RoS should cover the following 12 body systems and include specific details if mentioned, otherwise state 'denies any issues':\n\n"

                "2. Skin: Any skin issues or rashes.\n"
                "3. Head: Any headaches or head injuries.\n"
                "4. Eyes: Vision problems or eye pain.\n"
                "5. Ears, Nose, Throat: Hearing issues, nasal congestion, sore throat, etc.\n"
                "6. Cardiovascular: Heart problems, chest pain, etc.\n"
                "7. Respiratory: Breathing issues, cough, etc.\n"
                "8. Gastrointestinal: Abdominal pain, nausea, etc.\n"
                "9. Genitourinary: Urination problems, etc.\n"
                "10. Musculoskeletal: Muscle pain, joint pain, etc.\n"
                "11. Neurological: Numbness, tingling, etc.\n"
                "12. Psychiatric: Depression, anxiety, etc.\n\n"
                "Conversation Transcript:\n{transcript}\n\n"
                "Relevant Knowledge Base:\n{knowledge_base}\n\n"
                "Please ensure the RoS is comprehensive, clear, and formatted properly."
            )
        )

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-123-a3a58f04f90c>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)




Review of Systems:

1. General: The patient denies any general symptoms such as fatigue, weight changes, or fever.

2. Skin: The patient denies any skin issues or rashes.

3. Head: The patient denies any headaches or head injuries.

4. Eyes: The patient denies any vision problems or eye pain.

5. Ears, Nose, Throat: The patient denies any hearing issues, nasal congestion, or sore throat.

6. Cardiovascular: The patient denies any heart problems or chest pain.

7. Respiratory: The patient denies any breathing issues or cough.

8. Gastrointestinal: The patient reports experiencing abdominal pain after eating fried Chinese food, which started as a twinge in the middle of their abdomen and moved to the right side. The pain is described as sharp and has been worsening throughout the day. The patient also reports feeling nauseated and having a little bit of diarrhea. They deny any urinary symptoms.

9. Genitourinary: The patient denies any urination problems.

10. Musculoskeletal: The pati

In [146]:
from abc import ABC, abstractmethod
import json
import pandas as pd
from llama_index.core import Document, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI
from langchain.prompts import PromptTemplate
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-HwFPBND26SnyzXe8nuXgT3BlbkFJ4CQ71wBNBpziW0et7UGh"

# Abstract summarizer class
class AbstractSummarizer(ABC):
    @abstractmethod
    def __init__(self, knowledge_base_path):
        pass

    @abstractmethod
    def summarize(self, transcript_path):
        pass

# Hpisummary class
class Hiisummary(AbstractSummarizer):
    def __init__(self, knowledge_base_path):
        self.knowledge_base_path = knowledge_base_path
        self.knowledge_base = self.load_knowledge_base()
        self.index = self.create_index()

    def load_knowledge_base(self):
        # Loading the CSV file as a string
        df = pd.read_csv(self.knowledge_base_path)
        return df.to_string()

    def create_index(self):
        # Creating a document list
        documents = [Document(text=self.knowledge_base)]

        # Initializing required components for ServiceContext
        prompt_helper = PromptHelper(chunk_size_limit=4096, num_output=512, chunk_overlap_ratio=0.2)
        service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)

        # Creating and returning the index
        return GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

    def search_index(self, query):
        # Searching for relevant documents in the index
        results = self.retrieve_relevant_documents(query)
        return results

    def retrieve_relevant_documents(self, query):
        # Retrieve documents from the index using the query
        retriever = self.index.as_retriever(verbose=True)
        search_results = retriever.retrieve(query)

        # Combine the texts of the top results, limiting the total length
        combined_text = ""
        for result in search_results:
            combined_text += result.text.replace("Patient", "Other Patient") + "\n"
            if len(combined_text.split()) > 3000:  # Adjust this limit as needed
                break

        return combined_text

    def summarize(self, transcript_path):
        # Loading the transcript JSON file
        with open(transcript_path, 'r') as file:
            transcript_data = json.load(file)

        # Extracting the conversation text
        docs = [item['Alternatives'][0]['Content'] for item in transcript_data['Conversation']['TranscriptItems'] if 'Alternatives' in item and item['Alternatives']]
        transcript_text = " ".join(docs)

        # Retrieving relevant knowledge base entries
        relevant_knowledge_base = self.search_index(transcript_text)

        prompt_template = PromptTemplate(
            input_variables=["transcript", "knowledge_base"],
            template=(
                "Based on the provided clinical conversation and the Conversation Transcript, generate a detailed Review of Systems (RoS) for the patient based on Conversation Transcript. "
                "The RoS should cover the following 12 body systems and include specific details if mentioned, otherwise state 'denies any issues':\n\n"
                "Write Ros based on the transcript."

                "1. General: Chief complaint\n"
                "2. Skin: Any skin issues or rashes.\n"
                "3. Head: Any headaches or head injuries.\n"
                "4. Eyes: Vision problems or eye pain.\n"
                "5. Ears, Nose, Throat: Hearing issues, nasal congestion, sore throat, etc.\n"
                "6. Cardiovascular: Heart problems, chest pain, etc.\n"
                "7. Respiratory: Breathing issues, cough, etc.\n"
                "8. Gastrointestinal: Abdominal pain, nausea, etc.\n"
                "9. Genitourinary: Urination problems, etc.\n"
                "Conversation Transcript:\n{transcript}\n\n"
                "Relevant Knowledge Base:\n{knowledge_base}\n\n"
                "Please ensure the RoS is comprehensive, clear, and formatted properly."
                "Please go through the entire trancript and mention yes only if the pain is present in the transcript."
            )
        )

        # Generating the summary using the LLM
        llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        prompt = prompt_template.format(transcript=transcript_text, knowledge_base=relevant_knowledge_base)
        summary = llm(prompt)
        return summary

# Execute function
def execute(transcript_path, knowledge_base_path):
    # Generating the summary
    summarizer = Hiisummary(knowledge_base_path)
    return summarizer.summarize(transcript_path)

# Example usage
transcript_path = '/content/transcript.json'
knowledge_base_path = '/content/MTS-Dialog-TestSet-1-MEDIQA-Chat-2023.csv'
summary = execute(transcript_path, knowledge_base_path)
print(summary)


<ipython-input-146-771fa79a2a67>:40: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(prompt_helper=prompt_helper)




Review of Systems (RoS):

1. General: The patient presents with a chief complaint of severe stomach pain. Denies any fever, chills, weight loss, or fatigue.
2. Skin: Denies any skin issues or rashes.
3. Head: Denies any headaches or head injuries.
4. Eyes: Denies any vision problems or eye pain.
5. Ears, Nose, Throat: Denies any hearing issues, nasal congestion, sore throat, or difficulty swallowing.
6. Cardiovascular: Denies any heart problems or chest pain.
7. Respiratory: Denies any breathing issues or cough.
8. Gastrointestinal: Chief complaint of severe stomach pain. Reports consuming fried Chinese food last night. Pain started in the middle and moved to the right side. Rate pain at a 6/10. Reports nausea, but denies vomiting. Also reports a little bit of diarrhea. Denies any changes in stool color.
9. Genitourinary: Denies any urinary symptoms such as burning, frequency, urgency, or blood.
10. Musculoskeletal: Denies any joint pain or injuries.
11. Neurological: Denies any head